In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.es import random_opt
from src.es import evolutionary_strategy
from src.grad_opt import ADAM_opt
from src.api_helper import api_utils
from src.api_helper import env
import copyreg

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [23]:
## multi-process has problem with decorator
# def af(query): return env.rosenbrock(query); api = api_utils.multi_process_transform(af) 

@api_utils.transform
def api(query): return env.rosenbrock(query)

# @api_utils.multi_thread_transform
# def api(query): return env.rosenbrock(query)

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query:", x0); print(f"init reward: {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu



init query: tensor([[0.7851, 0.4338]])
init reward: tensor([[-1.]]), origin reward: 3.38


##### OPT

In [24]:
# hyperparameters
T = 10  # total number of iterations

adam = ADAM_opt(); ram_opt = random_opt(); es = evolutionary_strategy()

In [25]:
%%time
ram_opt.outer_loop(T, (-1,1), x0, r0, api, batch_size=2)

CPU times: user 7.72 ms, sys: 2.51 ms, total: 10.2 ms
Wall time: 10.1 ms


(tensor([[ 0.7851,  0.4338],
         [-0.7076, -0.3452],
         [ 0.8915,  0.7003],
         [-0.6418, -0.0364],
         [ 0.7742, -0.1271],
         [-0.0580, -0.0178],
         [ 0.5339, -0.6366],
         [ 0.7166,  0.5964],
         [ 0.4342, -0.0336],
         [ 0.4353,  0.9186],
         [ 0.1824, -0.4164],
         [ 0.0124,  0.7944],
         [ 0.5436,  0.0017],
         [ 0.8805,  0.9492],
         [ 0.0649, -0.8682],
         [ 0.4864,  0.6199],
         [-0.9395, -0.7662],
         [ 0.2736,  0.6889],
         [-0.6255,  0.3513],
         [-0.8264,  0.4318],
         [ 0.6581, -0.4906]]),
 tensor([ -1.0000, -22.0493,  -0.2671,  -6.7469, -15.6414,  -0.3447, -25.2135,
          -0.2270,  -1.5565, -15.8341,  -6.1847, -18.9651,  -2.6166,  -0.9000,
         -22.7946,  -4.4290, -81.6088, -11.3209,  -0.8294,  -2.8553, -25.2940]))

In [26]:
%%time
adam.outer_loop(T, x0, r0, api, env.rosenbrock_grad)

CPU times: user 10.5 ms, sys: 1.2 ms, total: 11.7 ms
Wall time: 11.1 ms


(tensor([[0.7851, 0.4338],
         [0.7841, 0.4348],
         [0.7831, 0.4358],
         [0.7821, 0.4368],
         [0.7811, 0.4378],
         [0.7801, 0.4388],
         [0.7791, 0.4398],
         [0.7781, 0.4408],
         [0.7771, 0.4418],
         [0.7761, 0.4428]]),
 tensor([-1.0000, -0.9726, -0.9455, -0.9189, -0.8927, -0.8670, -0.8417, -0.8168,
         -0.7923, -0.7683]))

RuntimeError: The expanded size of the tensor (0) must match the existing size (10) at non-singleton dimension 0.  Target sizes: [0].  Tensor sizes: [10]